In [ ]:
!pip install azure-cognitiveservices-anomalydetector

In [ ]:
from azure.cognitiveservices.anomalydetector import AnomalyDetectorClient
from azure.cognitiveservices.anomalydetector.models import Request, Point, Granularity
from msrest.authentication import CognitiveServicesCredentials
import pandas as pd
import json
import pytz

config = json.load(open("config.json"))

key = config["key"]
endpoint = config["endpoint"]

In [ ]:
client = AnomalyDetectorClient(endpoint, CognitiveServicesCredentials(key))

In [ ]:
df = pd.read_csv("energy_hourly.csv", delimiter=',', parse_dates=["Datetime"])
df.head()

In [ ]:
df = df.sort_values(by=["Datetime"])
df.head()

In [ ]:
df["Datetime"] = df["Datetime"].dt.tz_localize(pytz.utc).dt.strftime('%Y-%m-%dT%H:%M:%SZ')
df.head()

In [ ]:
series = []
for index, row in df.iterrows():
    series.append(Point(timestamp=row[0], value=row[1]))

In [ ]:
request = Request(series=series[0:3000], granularity=Granularity.hourly)

In [ ]:
response = client.entire_detect(request)

In [ ]:
True in response.is_anomaly

In [ ]:
response.is_anomaly

In [ ]:
for i in range(len(response.is_anomaly)):
    if response.is_anomaly[i]:
        print(i, df.iloc[i]["Energy"], df.iloc[i]["Datetime"])